# Instrukcja - Transformacja Hougha

### Cel:
- zapoznanie z transformacją Hougha dla pojedynczego punktu,
- kilku punktów, prostych figur
- wykorzystanie transformacji Hougha do detekcji linii prostych na rzeczywistym obrazie
- transformacja Hougha w przestrzeni ab

### Transformacja Hough'a

Transformacja Hougha dla prostych jest metodą detekcji współliniowych punktów. Każda prosta może być jednoznacznie przedstawiona za pomocą dwóch parametrów. Przestrzeń tych parametrów to przestrzeń Hougha. Najczęściej wykorzystywanymi parametrami w tej metodzie są współczynniki ρ,θ

opisujące równanie prostej w postaci normalnej:

ρ=x⋅cos(θ)+y⋅sin(θ)

gdzie: ρ - promień wodzący, θ - kąt pomiędzy ρ a osią OX.

Własności transformacji Hougha:
- prostej w przestrzeni kartezjańskiej odpowiada punkt w przestrzeni Hougha
- pękowi prostych przechdzących przez punkt w przestrzeni kartezjańskiej odpowiada krzywa sinusoidalna w przestrzeni Hougha
- punkty leżące na tej samej prostej (w przestrzeni kartezjańskiej) korespondują z sinusoidami przechodzącymi przez wspólny punkt w przestrzeni Hougha.

Metoda wyliczania transformacji Hougha składa się z następujących kroków:
- przez każdy badany (różny od zera) punkt obrazu prowadzony jest pęk prostych, przechodzących przez ten punkt
- każda z tych prostych transformowana jest do przestrzeni Hougha i tworzy tam punkt o współrzędnych ρ,θ
- w ten sposób, każdy punkt obrazu pierwotnego (pęk prostych) jest odwzorowany w sinusoidalną krzywą w przestrzeni Hougha

Przestrzeń Hougha jest przestrzenią akumulacyjną tzn. punkty sinusoidalnych krzywych, wygenerowanych dla punktów obrazu pierwotnego dodają się w miejscach, w których krzywe te przecinają się. Powstałe w ten sposób (w przestrzeni Hougha) maksima odpowiadają zbiorom punktów, należących do jednej prostej. Współrzędne ρ,θ
tego maksimum jednoznacznie określają położenie prostej na obrazie pierwotnym.

### Transformacja Hougha dla małej liczby punktów.
   1. Uruchom poniższy kod. W tablicy `im` wskaż jeden punkt, dla którego ma zostać obliczona transformata.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
import os

if not os.path.exists("kwadraty.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/kwadraty.png --no-check-certificate
if not os.path.exists("lab112.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/lab112.png --no-check-certificate
if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/dom.png --no-check-certificate

im = np.zeros((64,64), dtype=np.uint8)

im[18, 31] = 1

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
ax.imshow(im, 'gray')
ax.axis('off')
plt.show()

3. Wykonaj transformację Hougha obazu im. Wykorzystaj funkcję *hough_line* z modułu _skimage.transform_. Funkcja zwraca: macierz H (przestrzeń Hougha) oraz dwa wektory theta i rho dla kolejnych 
4. Wyświetl przestrzeń Hougha za pomocą funkcji _plt.imshow_ (można też wykorzystać poniższą funkcję *show_hough*). Jak "wygląda" pojedynczy punkt w przestrzeni Hougha?

In [ ]:
def show_hough(h, image):
    # Generating figure 1
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ax = axes.ravel()

    ax[0].imshow(image, 'gray')
    ax[0].set_title('Input image')
    ax[0].set_axis_off()

    ax[1].imshow(h, 'gray')
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')
    
    plt.tight_layout()
    plt.show()    

hough, theta, rho = hough_line(im)
show_hough(hough, im)

#### Ad 4)

Pojednyczy punkt to sinusoida.

5. Powtórz punkty 1-4, ale tym razem klinkij dwa punkty. Jak zmienia się przestrzeń Hougha?
6. Powtórz punkty 1-4, ale tym razem kliknij kilka punktów starając się aby były współliniowe. Zaobserwuj zmiany w przestrzeni Hougha
7. Poeksperymentuj z różnymi układami punktów

In [ ]:
# 5

im = np.zeros((64,64), dtype=np.uint8)
im[52, 42] = 1
im[15, 21] = 1
hough, theta, rho = hough_line(im)
show_hough(hough, im)

In [ ]:
# 6

im = np.zeros((64,64), dtype=np.uint8)
im[34, 10] = 1
im[34, 24] = 1
im[34, 32] = 1
im[34, 59] = 1
hough, theta, rho = hough_line(im)
show_hough(hough, im)

In [ ]:
# 7

im = np.zeros((64,64), dtype=np.uint8)
im[41, 45] = 1
im[18, 54] = 1
im[31, 59] = 1
im[3, 43] = 1
im[12, 15] = 1
im[10, 47] = 1
hough, theta, rho = hough_line(im)
show_hough(hough, im)

### Transformata Hougha dla pojedynczego obiektu

W tym podpunkcie pokazane zostanie praktycznie wykorzystanie transformaty Hougha - do detekcji prostych na sztucznym rysunku.

   1. Wczytaj obraz "kwadraty.png". Wyświetl go.
   2. Wykonaj detekcję krawędzi jedną z metod gradientowych. Ważne aby obraz krawędzi był jak najlepszej jakości - co oznacza cienkie (nawet niekoniecznie ciągłe) krawędzie - dla tego przypadku nie powinno być trudne do uzyskania. Wyświetl obraz po detekcji krawędzi.
   3. Wykonaj transformatę Hougha obrazu krawędziowego. Wykorzystaj funkcję *hough\_line*.
   4. Wyświetl macierz H. Czy widoczna jest taka liczba maksimów jakiej się spodziewamy?

In [ ]:
img_kwadraty = cv2.imread("kwadraty.png", cv2.IMREAD_GRAYSCALE)
img_kwadraty_edges = cv2.Canny(img_kwadraty, 100, 200)
hough, theta, rho = hough_line(img_kwadraty)

plt.figure(figsize=(12, 5))
plt.subplot(131)
plt.imshow(img_kwadraty, 'gray')
plt.title('Obraz oryginalny')
plt.axis('off')
plt.subplot(132)
plt.imshow(img_kwadraty_edges, 'gray')
plt.title('Krawędzie')
plt.axis('off')
plt.subplot(133)
plt.imshow(hough, 'gray')
plt.title('Macierz H')
plt.show()

 5. W module skimage.transform dostępna jest funkcja do automatycznej analizy przestrzeni Hougha - wyszukiwania maksimów - *hough\_line\_peaks*. Jako parametry przyjmuje ona wyniki funkcji *hough\_line* (macierz H, theta i rho). Dodatkowo można podać próg powyżej którego punkt uznawany jest za maksimum (_threshold_ - domyslnie jest to połowa maksimum w przestrzeni H) oraz liczbę poszukiwanych maksimów (*num_peaks*). Funkcja zwraca współrzędne maksimów. Wykorzystaj funkcję *hough\_line\_peaks* do znalezienia maksimów odpowiadających krawędziom kwadratów.
 6. Wyświetl macierz H używając konstrukcji:

In [ ]:
max_value, max_theta, max_rho = hough_line_peaks(hough, theta, rho)
fig, ax = plt.subplots(1, figsize=(15,15))
ax.set_aspect('equal')
ax.imshow(hough)

max_theta += np.pi/2
max_theta = np.rad2deg(max_theta)
max_rho += hough.shape[0] / 2
plt.imshow(hough, 'gray')

for v, t, r in zip(max_value, max_theta, max_rho):
    c = plt.Circle((t, r), 5, color='yellow', fill=False)
    ax.add_patch(c)
    print(f"Punkty ({t:.3f}, {r:.3f}): {v}")
plt.show()

#### Ad 4)

Występuje 8 maksimów - tego się spodziewaliśmy

Taki zapis pozwoli na dołożenie annotacji (okręgów) w miejscach znalezionych maksimów. Wyrysowanie okręgu w punkcie x, y (o rozmiarze 10, w czerwonym kolorze, bez wypełnienia środka) realizuje wywołanie: 

**circle = plt.Circle((x, y), 10, color='r', fill=False)**

natomiast dołożenie takiego okręgu do obrazu to:

**ax.add_patch(circle)**

Zaznacz maksima na obrazie wykorzystując rezultat funkcji *hough\_line\_peaks* biorąc pod uwagę, że zwraca ona kąty w radianach z przedziału od -pi/2 do pi/2, a rho z przedziału od -r/2 do r/2 gdzie r to pionowy rozmiar przestrzeni Hougha. 

7. Istnieje też możliwość przeprowadzenia transformacji Hougha z użyciem biblioteki OpenCV. W bibliotece znajdują się dwie wersje funkcji wyszukującej linie proste - 'klasyczna' - _HoughLines_ oraz probabilistyczna _HoughLinesP_. Zadna z nich nie zwraca przestrzeni Hougha. Wynikiem działania pierwszej jest lista parametrów prostych (krotki zawierające rho, theta). Druga zwraca krotki 4-ro elementowe ze współrzędnymi końców odcinków wykorzystanych do wylicznia parametrów (czyli znalezienia prostej). 
8. Wyznacz linie obecne na obrazie za pomocą funkcji _HoughLines_. Wykryte linie wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). Do wyświetlania linii wykorzystaj przykładowy kod:

In [ ]:
hough_lines = cv2.HoughLines(img_kwadraty_edges, 1, np.pi/180, 50)
img_kwadraty_color = cv2.imread("kwadraty.png")

for h_line in hough_lines:
    rho, theta = h_line[0, 0], h_line[0, 1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))  
    img_kwadraty_color = cv2.line(img_kwadraty_color, (x1, y1), (x2, y2), [0, 255, 0], thickness=2)

plt.figure(figsize=(8,8))
plt.imshow(img_kwadraty_color)
plt.axis('off')
plt.title("Otrzymany rezultat")
plt.show()

9. Wyznacz odcinki obecne na obrazie za pomocą funkcji _HoughLinesP_. Wykryte odcinki wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). 

In [ ]:
img_kwadraty_color = cv2.imread("kwadraty.png")
img_kwadraty_color_edges = cv2.Canny(img_kwadraty_color, 100, 200)
hough_lines = cv2.HoughLinesP(img_kwadraty_color_edges, 1, np.pi/180, 10)
for h_line in hough_lines:
    x1, y1, x2, y2 = h_line[0]
    img_kwadraty_color = cv2.line(img_kwadraty_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
plt.figure(figsize=(8,8))
plt.imshow(img_kwadraty_color)
plt.axis('off')
plt.title("Otrzymany rezultat")
plt.show()

### Transformata Hougha dla obrazu rzeczywistego.

Bazując na kodzie stworzonym w punkcie B wyszukamy linie na obrazie rzeczywistym.
   1. Wczytaj obraz "lab112.png". Wyświetl go.
   2. Wykorzystując wszystkie poznane techniki przetwarzania obrazów (filtracja, przekształcenia morfologiczne, binaryzację, detekcję krawędzi) wyodrębnij krawędzie samych kwadratów - tak aby były jak najlepszej jakości (cienkie) - jednocześnie eliminując z obrazu zakłócenia.
   3. Wykorzystaj funkcje *hough_line* i *hough_line_peaks* do detekcji linii na obrazie, a następnie np. wykorzystując kod z punktu 8 poprzedniego ustępu wyrysuj na oryginalnym obrazie znalezione linie.

In [ ]:
img_lab112 = cv2.imread('lab112.png', cv2.IMREAD_GRAYSCALE)
img_lab112_edges = cv2.Canny(cv2.GaussianBlur(img_lab112, (3, 3), 1), 130, 240)


plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(img_lab112, 'gray')
plt.axis('off')
plt.title("Obraz oryginalny")
plt.subplot(122)
plt.imshow(img_lab112_edges, 'gray')
plt.axis('off')
plt.title("Krawędzie")
plt.show()

In [ ]:
hough, theta, rho = hough_line(img_lab112_edges)
hough2, theta2, rho2 = hough_line_peaks(hough, theta, rho)
img_kwadraty_color = cv2.imread("lab112.png")

for t, r in zip(theta2, rho2):
    a = np.cos(t)
    b = np.sin(t)
    x0 = a*r
    y0 = b*r
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))  
    img_kwadraty_color = cv2.line(img_kwadraty_color, (x1, y1), (x2, y2), [0, 255, 0], thickness=2)
    
plt.figure(figsize=(8,8))
plt.imshow(img_kwadraty_color)
plt.axis('off')
plt.title("Otrzymany rezultat")
plt.show()

4. Wczytaj obraz "dom.png". Wypróbuj działanie transformacji Hougha na tym obrazie z wykorzystaniem funkcji _cv2.HoughLinesP_  (oczywiście po odpowiednich przekształceniach). Postaraj się tak przygotować obraz z krawędziami i dobrać parametry aby wyrysować na oryginalnym obrazie odcinki obejmujące zarysy domu. Weź pod uwage dodatkowe parametry funkcji, takie jak:   minLineLength, maxLineGap.

In [ ]:
img_house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
img_house_edges = cv2.Canny(cv2.GaussianBlur(img_house, (3, 3), 1), 130, 240)


plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(img_house, 'gray')
plt.axis('off')
plt.title("Obraz oryginalny")
plt.subplot(122)
plt.imshow(img_house_edges, 'gray')
plt.axis('off')
plt.title("Krawędzie")
plt.show()

In [ ]:
hough_lines = cv2.HoughLinesP(img_house_edges, rho=1, theta=np.pi/180, threshold=0, minLineLength=0, maxLineGap=0)
img_house_color = cv2.imread('dom.png')

for h_line in hough_lines:
    x1, y1, x2, y2 = h_line[0]
    img_house_color = cv2.line(img_house_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
plt.figure(figsize=(8,8))
plt.imshow(img_house_color)
plt.axis('off')
plt.title("Otrzymany rezultat")
plt.show()